In [50]:
import urllib
from urllib.request import urlopen
import json
import time
import requests
import pandas as pd
from datetime import datetime as dt
from requests.auth import HTTPBasicAuth
from IPython.display import display, HTML, clear_output


# Python 3: Jupyter Notebook

#Public API
class Sfox:
        
    def __init__(self, config={'endpoint': "api.sfox.com", 'version': "v1"}):
        self.endpoint = config['endpoint']
        self.apiVersion = config['version']

    def urlFor(self,resource):
        return "https://" + self.endpoint + "/" + self.apiVersion + "/" + resource

    def rawbook(self,market):
        ret = urlopen(self.urlFor("markets/orderbook/")+ market)
        return json.loads(ret.read())
    
    def liveorderbook(self, market):
        while True:
            timestamp=(str(dt(time.localtime().tm_year, time.localtime().tm_mon, time.localtime().tm_mday, 
                              time.localtime().tm_hour, time.localtime().tm_min, time.localtime().tm_sec)))
            price= Sfox.rawbook(market)
            buyers= pd.DataFrame(price['bids'], columns=['Bid Price','Bid Size','Bid Exchange'])
            buyers['Bid Notional']=buyers['Bid Price']*buyers['Bid Size']
            sellers= pd.DataFrame(price['asks'], columns=['Sell Price','Sell Size','Sell Exchange'])
            sellers['Sell Notional']=sellers['Sell Price']*sellers['Sell Size']

            orderbook= pd.concat([buyers,sellers], axis=1)
            orderbook=orderbook[['Bid Price','Bid Size','Sell Price','Sell Size','Bid Exchange','Bid Notional',
                             'Sell Exchange', 'Sell Notional']]
            print("Orderbook:",market, "Time:", timestamp)
            display(HTML(orderbook.to_html()))
            time.sleep(1)
            clear_output(wait=True)
   
    #split GetBestPrice from API reference page (https://www.sfox.com/developers/#price-data)
    def bestBuyPrice(self,quantity):
        ret = urlopen(self.urlFor("offer/buy?amount=")+ str(quantity))
        print("Buy")
        return json.loads(ret.read())

    def bestSellPrice(self,quantity):
        ret = urlopen(self.urlFor("offer/sell?amount=")+ str(quantity))
        print("Sell")
        return json.loads(ret.read())
        
        
        

#Private API
    
class PrivateSfox():

    def __init__(self, APIKey, config={'endpoint': "api.sfox.com", 'version': "v1"}):
        self.endpoint = config['endpoint']
        self.apiVersion = config['version']
        self.APIKey = APIKey

    def urlFor(self,resource):
        return "https://" + self.endpoint + "/" + self.apiVersion + "/" + resource
    
    def market_buy(self,quantity,pair):
        ret = requests.post(self.urlFor("orders/buy"), data={'quantity': str(quantity),'currency_pair':str(pair)}, 
                            auth=HTTPBasicAuth(self.APIKey,''))
        return json.loads(ret.text)
    
    def market_sell(self,quantity,pair):
        ret = requests.post(self.urlFor("orders/sell"), data={'quantity': str(quantity),'currency_pair':str(pair)}, 
                            auth=HTTPBasicAuth(self.APIKey,''))
        return json.loads(ret.text)
    
    def limit_buy(self,quantity,price,pair):
        ret = requests.post(self.urlFor("orders/buy"), data={'quantity': str(quantity), 'price': str(price),
                            'currency_pair':str(pair)}, auth=HTTPBasicAuth(self.APIKey,''))
        return json.loads(ret.text)
    
    def limit_sell(self,quantity,price,pair):
        ret = requests.post(self.urlFor("orders/sell"), data={'quantity': str(quantity), 'price': str(price),
                            'currency_pair':str(pair)}, auth=HTTPBasicAuth(self.APIKey,''))
        return json.loads(ret.text)
    
        
    #Account API
    
  
    def balance(self):
        ret = requests.get(self.urlFor("user/balance"), auth=HTTPBasicAuth(self.APIKey,''))
        return json.loads(ret.text)
    
# jsfour: just make the function "balance". Also Id return the raw data here and let someone put it into pandas if they want to.
#         balance=json.loads(ret.text)
#         balance=pd.DataFrame(balance,columns=['currency','available','balance','held'])
#         portfolio=balance.set_index('currency')
#         return portfolio

    def getOrderStatus(self,order_id):
        ret = requests.get(self.urlFor("order/")+ str(order_id), auth=HTTPBasicAuth(self.APIKey,''))
        return json.loads(ret.text)
    
    def getTradeHistory(self):
        ret = requests.get(self.urlFor("account/transactions?limit=250&offset=0"), auth=HTTPBasicAuth(self.APIKey,''))
        return json.loads(ret.text)
    
    
# jsfour: "getTradeHistory"- Id return the raw data here and let someone put it into pandas if they want to.
#         trades=json.loads(ret.text)
#         print("Raw API:\n", trades)
#         trade_hist=pd.DataFrame(trades, columns=['action','price','amount','currency','day','fees','id'])
#         return trade_hist
    
    def getActiveOrders(self):
        ret = requests.get(self.urlFor("orders"), auth=HTTPBasicAuth(self.APIKey,''))
        return json.loads(ret.text)
    
    
    def cancelOrder(self,order_id):
        ret = requests.delete('https://api.sfox.com/v1/order/'+ str(order_id), auth=HTTPBasicAuth(self.APIKey,''))
        order=self.getOrderStatus(order_id)
        if order['status'] == 'Canceled':
            print("Order Canceled")
        else:
            print("Did NOT Cancel")
        return json.loads(ret.text)
    
    def cancelLastOrder(self):
        orders=self.getActiveOrders()
        last_order=orders[0]['id']
        self.cancelOrder(last_order)
        return
    
    def cancelAllOrders(self):
        order=self.getActiveOrders()
        ids=[]
        print(len(order),"orders cancelled")
        for i in range(len(order)):
            ids.append(order[i]['id'])
        for i in range(len(ids)):
            self.cancelOrder(ids[i])
    
        
    
#Algorithms/Trade API 
    
    # Talk to George/Akbar/Alex but im not sure if we want to include the algos below.  
        
class Strategy():   
    
    def __init__(self, APIKey, config={'endpoint': "api.sfox.com", 'version': "v1"}):
        self.endpoint = config['endpoint']
        self.apiVersion = config['version']
        self.APIKey = APIKey
    
    def arb_snipe(self, market, show=False):
        price= Sfox().rawbook(market)
        buyers= pd.DataFrame(price['bids'], columns=['Bid Price','Bid Size','Bid Exchange'])
        buyers['Bid Notional']=buyers['Bid Price']*buyers['Bid Size']
        sellers= pd.DataFrame(price['asks'], columns=['Sell Price','Sell Size','Sell Exchange'])
        sellers['Sell Notional']=sellers['Sell Price']*sellers['Sell Size']

        orderbook= pd.concat([buyers,sellers], axis=1)
        orderbook['Sell Fee (per btc)']=(orderbook['Sell Price']*.0025)
        orderbook['Buy Fee (per btc)']=(orderbook["Bid Price"]*.0025)
        orderbook["Arb $ (per btc)"]= orderbook["Bid Price"]-orderbook['Sell Price'] 
        
        arb_orderbook = orderbook[orderbook['Arb $ (per btc)'] > orderbook['Buy Fee (per btc)']+orderbook['Sell Fee (per btc)']]

        while len(arb_orderbook) > 0 :
            print("Arb exists")
            order_size= min(orderbook['Sell Size'][0],orderbook['Bid Size'][0])
            buy_side=min(orderbook['Bid Price'][0],orderbook['Sell Price'][0])
            sell_side=max(orderbook['Bid Price'][0],orderbook['Sell Price'][0])
            print("Selling",sell_side-orderbook['Sell Fee'][0])
            print("Buying",buy_side+orderbook['Buy Fee'][0])
            print("Arbitrage after fees (per btc):", sell_side-orderbook['Sell Fee'][0]-buy_side-orderbook['Buy Fee'][0])
            self.market_buy(order_size)
            self.market_sell(order_size)

        else:
            print("No Arb opportunities")
        
        if show==True:
            display(HTML(orderbook.to_html()))

    
    def live_spread_strategy(self,beta,market1,market2, Trade=False):
        
        balance=PrivateSfox(self.APIKey).balance()
        balance=pd.DataFrame(balance,columns=['currency','available','balance','held'])
        port=balance.set_index('currency')
        
        holdings={}
        for cryptos in port.index:
            holdings[cryptos]=port.loc[cryptos]['balance']

        while True:
            timestamp=(str(dt(time.localtime().tm_year, time.localtime().tm_mon, time.localtime().tm_mday, 
                              time.localtime().tm_hour, time.localtime().tm_min, time.localtime().tm_sec)))
            buy_mkt1=Sfox().rawbook(market1)['asks']
            sell_mkt1=Sfox().rawbook(market1)['bids']
            buy_mkt2=Sfox().rawbook(market2)['asks']
            sell_mkt2=Sfox().rawbook(market2)['bids']


            spread= pd.DataFrame((buy_mkt1), columns=[str(market1)+' Asks','Size','Exchange'])
            spread2= pd.DataFrame((sell_mkt1), columns=[str(market1)+' Bids','Size','Exchange'])
            spread3= pd.DataFrame((buy_mkt2), columns=[str(market2)+' Asks','Size','Exchange'])
            spread4= pd.DataFrame((sell_mkt2), columns=[str(market2)+' Bids','Size','Exchange'])

            spreads=pd.concat([spread,spread2,spread3,spread4],axis=1)
            spreads['Buy Spread']=spreads[str(market1)+' Asks']-spreads[str(market2)+' Bids']*beta
            spreads['Sell Spread']=spreads[str(market1)+' Bids']-spreads[str(market2)+' Asks']*beta
            print("Inventory", holdings)
            print("Time:", timestamp)
            display(HTML(spreads.to_html()))
            time.sleep(0.5)
            clear_output(wait=True)
        if Trade==True:
            print('Run your spread strategy: Place execution strategy here')
